## Librairies et GPU

In [1]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=11fdeeea61c36815589ed642bddfd248ccec2c9934808454317beb7473db07b2
  Stored in directory: /root/.cache/pip/wheels/b1/e7/99/2b32600270cf23194c9860f029d3d5db075f250bc39028c045
Successfully built GPUtil


In [2]:
## Librairies

from transformers import BertConfig,BertForMaskedLM, AutoModelForMaskedLM
from transformers import AutoTokenizer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd 
import copy
from datasets import load_metric

import random
import shutil
from tqdm import tqdm
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import time
import gc


In [3]:
## GPU
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.empty_cache()

## Fonctions Utiles

In [4]:
## Fonctions utiles

# Masquage de 15% d'un texte
def Masks(Tokens):
  L_indice=torch.zeros_like(Tokens).to(device)
  tokenised_new=copy.deepcopy(Tokens) ## Masques 

  for i in range (len(Tokens)):
      for proba in range (int(0.15 * float(Tokens[i].tolist().index(102)))):
          mask_indice=random.randrange(start=1, stop=Tokens[i].tolist().index(102)-1)
          tokenised_new[i][mask_indice]=103
          L_indice[i][mask_indice]=1
  return(tokenised_new,L_indice)

# Création Matrice Pseudo-Attention (prend en compte les mots voisins à poids égals)
def matrice_pseudo_attention_cbow(weight,window_size,L_size):
    window_size1=np.c_[np.zeros(L_size) ,weight*np.identity(L_size)[:,:-1] ] + weight*np.identity(L_size) + weight*np.c_[np.identity(L_size)[:,1:], np.zeros(L_size) ]
    window_size2=window_size1+np.c_[np.zeros(L_size),np.c_[np.zeros(L_size),np.identity(L_size)[:,:-2]]]
    window_size3=window_size2 + np.c_[np.zeros(L_size),np.c_[np.zeros(L_size),np.c_[np.zeros(L_size),np.identity(L_size)[:,:-3]]]]
    window_size4=window_size3 + np.c_[np.zeros(L_size),np.c_[np.zeros(L_size),np.c_[np.zeros(L_size),np.c_[np.zeros(L_size),np.identity(L_size)[:,:-4]]]]]
    window_size5=window_size4 + np.c_[np.zeros(L_size),np.c_[np.zeros(L_size),np.c_[np.zeros(L_size),np.c_[np.zeros(L_size),np.c_[np.zeros(L_size),np.identity(L_size)[:,:-5]]]]]]
 
    if window_size==2:
        att=window_size2
    if window_size==3:
        att=window_size3
    if window_size==4:
        att=window_size4
    if window_size==5:
        att=window_size5
    else:
        att=window_size1  # par défaut aussi
    return(torch.from_numpy(att))

# Calcul de l'accuracy
def accuracy_score(Liste_de_logits,verite_terrain):
    score=0
    for i in range (len(verite_terrain)): #pour chaque mot masqué
        y_top=(-Liste_de_logits[i]).argsort()[:TOP]  #les mots les plus probables selon Y
        if verite_terrain[i] in y_top:
            score+=1
    return(score/len(verite_terrain))

#Passer des probas en pourcentage

def MultiplyPourcent(Liste):
    for i in range (len(Liste)):
        Liste[i]=Liste[i]*100
    return(Liste)

def DividePourcent(Liste):
    for i in range (len(Liste)):
        Liste[i]=Liste[i]/100
    return(Liste)


## Paramètres Généraux

In [5]:
## Paramètres

MAX_SEQ_LEN = 512
LENGTH_BATCH=10000
BATCH_SIZE=10
TOP=5
epochs=20
window_size=5 #Fenêtre du contexte: 1= 1 avant 1 après,2= 1 avant 2 après,  3= #2 avant 2 après,  4= #2 avant 3 après, 5= #3 avant 3 après

checkpoint="bert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Chargement et Traitement du Dataset

In [6]:
## Chargement et traitement du dataset 

with open("/kaggle/input/wikipedia-corpus/train (1).txt") as my_file:
    Input=my_file.read()

Liste_index=[]
Input=Input.split('\n')
for i in range (len(Input)):
    if len(Input[i])>500:
        Liste_index.append(i)

Input_processed=[]
for i in range (0,len(Liste_index)-3,3):
    string= " ".join([Input[Liste_index[i]], Input[Liste_index[i+1]]])
    string= " ".join([string, Input[Liste_index[i+2]]])
    Input_processed.append(string)
    
Input_sentences=Input_processed[:int(0.7*len(Input_processed))]
Test_sentences=Input_processed[int(0.7*len(Input_processed)):]

print(len(Input_sentences))
print(len(Test_sentences))

Input_sentences[0]

2433
1044


"Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism. Humans lived in societies without formal hierarchies long before the establishment of formal states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist thought are found throughout history, modern anarchism emerged from the Enlightenment. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement flourished in 

## Tokeniser et Masquages des mots

In [7]:
## Tokeniser

Input_tokenised=tokenizer.batch_encode_plus(Input_sentences,return_tensors="pt",padding="max_length", max_length=MAX_SEQ_LEN, truncation=True).to(device)
Test_tokenised=tokenizer.batch_encode_plus(Test_sentences,return_tensors="pt",padding="max_length", max_length=MAX_SEQ_LEN, truncation=True).to(device)

In [8]:
## Masking

# Input_tokenised_masked,L_indice=Masks(Input_tokenised)
# Test_tokenised_masked,L_indice2=Masks(Test_tokenised)

## Notre modèle

In [9]:
## Modèle

class MLM(nn.Module):
    def __init__(self,bert):
        super(MLM, self).__init__()

        self.embedding= nn.Embedding(30522,768)

        self.v = nn.Linear(768, 30522,bias = False)
        
        self.v.weight = self.embedding.weight

#         self.v.weight = torch.nn.Parameter(model.bert.embeddings.word_embeddings.weight,requires_grad=True)

    def forward(self, tokens):
        emb_init = self.embedding(tokens)
        A=matrice_pseudo_attention_cbow(1,window_size,emb_init.shape[1]).to(device)
        emb_init_context=torch.matmul(A.float(),emb_init)
        return self.v(emb_init_context)

## Création des batchs

In [10]:
## Batchs des données 

class TextDataset(Dataset):

    def __init__(self, X):
        self.inputs = X
#         self.masques = M
#         self.verite_terrain=torch_verite_terrain

    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
#         return (self.inputs[index],self.masques[index],self.verite_terrain[index])
        return (self.inputs[index])


In [11]:
#Création des batchs

data_train= TextDataset(Input_tokenised["input_ids"])

# data_test= TextDataset(
#     Test_tokenised_masked["input_ids"],
#     L_indice2,
#     Test_tokenised["input_ids"])

train_loader = DataLoader(data_train,batch_size=BATCH_SIZE,shuffle=False)
    
# test_loader = DataLoader(data_test,batch_size=BATCH_SIZE,shuffle=False)

## Training

In [12]:
# ## Training de notre modèle

#Notre modèle
# cbow=MLM(model)
# model.to(device)
# cbow.to(device)

# Loss 
# CEL = nn.CrossEntropyLoss()
# MSE= nn.MSELoss()

#Optimizer
# lr=0.001
# optimizer = torch.optim.Adam(cbow.parameters(), lr )
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

# losses=[]
# total_accuracy_train=[]
# total_accuracy_test=[]

# start = time.time()
# for epoch in range(epochs):
#     cbow.zero_grad()
#     total_loss = 0
    
#     ## TRAIN
#     for batch in tqdm((train_loader)):

#         ## BATCH 
#         X,M,verite_terrain = batch
        
#         ## BERT ENTRAINE
#         with torch.no_grad():
#             Y_bert=model(X).logits   
        
#         ## MODEL
#         Y_model=cbow(X)                           
    
#         ## LOSS
#         loss_real=CEL(Y_model[M==1],F.one_hot(verite_terrain[M==1],num_classes=30522).to(torch.float32))
#         loss_bert=MSE(Y_model[M==1],Y_bert[M==1])
#         loss=loss_real + loss_bert
#         total_loss+= loss_real.item() + loss_bert.item() 

#         ## ACCURACY 
#         accuracy_train=accuracy_score(Y_model[M==1],verite_terrain[M==1])

#         loss.backward()
#         optimizer.step()

#     ## TEST
#     for batch_test in (test_loader):
#         X_test,M_test,verite_terrain_test = batch_test
        
#         ## MODEL
#         Y_model_test=cbow(X_test)

#         # ACCURACY 
#         accuracy_test=accuracy_score(Y_model_test[M_test==1],verite_terrain_test[M_test==1])

#     print("Precision train: " + str(round(accuracy_train*100.,2)) + "% ")
#     print("Precision test: " + str(round(accuracy_test*100.,2)) + "% ") 
 
#     losses.append(total_loss)
#     total_accuracy_train.append(accuracy_train)
#     total_accuracy_test.append(accuracy_test)

# end = time.time()
# time = end - start

In [ ]:
## Training Bert pas trained

#Bert pas entrainé (pour comparaison)
my_config = BertConfig()
model2 = BertForMaskedLM(my_config)
model.to(device)
model2.to(device)

# Loss
CEL=nn.CrossEntropyLoss()
MSE= nn.MSELoss()
losses=[]

#Optimizer
lr2=1e-4
optimizer = torch.optim.Adam(model2.parameters(), lr2 , betas=(0.9, 0.999))
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

# Steps
NUM_ACCUMULATION_STEPS= 16

# Accuracy
total_accuracy_train=[]
total_accuracy_test=[]

for epoch in range(epochs):
    model2.zero_grad()
    total_loss = 0
    
    ## TRAIN
    for idx,X in enumerate (tqdm(train_loader)):

        ## BATCH 
        Y,M = Masks(X)
        
        ## BERT PAS ENTRAINE
        Logits_bert_not_trained=model2(Y).logits
                
        ## BERT ENTRAINE
        with torch.no_grad():
            Logits_bert=model(X).logits

        ## LOSS
        loss_real=CEL(Logits_bert_not_trained[M==1],F.one_hot(X[M==1],num_classes=30522).to(torch.float32))
        loss_bert=MSE(Logits_bert_not_trained,Logits_bert)
        loss= loss_bert + loss_real 
        loss = loss / NUM_ACCUMULATION_STEPS
        
        total_loss =  loss_bert.item() + loss_real.item()

        loss.backward()
        
        if ((idx + 1) % NUM_ACCUMULATION_STEPS == 0) or (idx + 1 == len(train_loader)):
            optimizer.step()

        ## ACCURACY 
        accuracy_train=accuracy_score(Logits_bert_not_trained[M==1],X[M==1])
#         accuracy_train_bert=accuracy_score(Y_bert[M==1],X[M==1])
        total_accuracy_train.append(accuracy_train)
        
    gc.collect()
    torch.cuda.empty_cache()
    print("Precision train: " + str(round(sum(total_accuracy_train)/len(total_accuracy_train)*100.,2)) + "% ")

    losses.append(total_loss)


100%|██████████| 244/244 [04:06<00:00,  1.01s/it]


Precision train: 15.94% 


100%|██████████| 244/244 [04:04<00:00,  1.00s/it]


Precision train: 16.28% 


100%|██████████| 244/244 [04:04<00:00,  1.00s/it]


...

100%|██████████| 244/244 [04:04<00:00,  1.00s/it]


Precision train: 17.22% 


100%|██████████| 244/244 [04:04<00:00,  1.00s/it]


Precision train: 17.51% 


 58%|█████▊    | 141/244 [02:21<01:45,  1.02s/it]

In [ ]:
## Plot des Courbes de Trainings

plt.figure()
plt.subplot(2,1,1)
plt.plot(MultiplyPourcent(total_accuracy_train),'r', label='accuracy de train')
plt.plot(MultiplyPourcent(total_accuracy_test),'g', label='accuracy de test')
plt.legend()
plt.title('Accuracy train')
plt.xlabel("Nombres d'époques")
plt.ylabel("Accuracy (en %)")
plt.subplot(2,1,2)
plt.plot(losses)
plt.title('Loss fonction')
plt.xlabel("Nombres d'époques")
plt.ylabel("Mean square error")
plt.savefig("Loss.png")

In [ ]:
## Sauvegardes des paramètres 

torch.save(cbow, 'best-model.pt') 
nbr_parameters= sum([np.prod(p.size()) for p in cbow.parameters()])
print(nbr_parameters)
print(time)
parameters=np.array([[nbr_parameters],
                     [cbow],
                     [epochs],
                     [time],
                     [lr]])

lignes_param = ['Nombres de Parametres: ', 'Model','Epoques: ' ,'Training time','Learning rate: ']

param_df = pd.DataFrame(data = parameters,index = lignes_param)

param_df.to_csv('Paramètres' +str(Nom)  + '.csv',sep =';')

In [ ]:
# from transformers import BertConfig,BertForMaskedLM
# my_config = BertConfig()
# model2 = BertForMaskedLM(my_config)
# model2.bert.embeddings.word_embeddings.weight[10,1]

In [ ]:
# model.bert.embeddings.word_embeddings.weight[10,1]